In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str,
                         labels: np.ndarray) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.token_indexers import PretrainedBertIndexer

token_indexer = PretrainedBertIndexer(
    pretrained_model="bert-base-uncased",
    max_pieces=config.max_seq_len,
    do_lowercase=True,
 )
# apparently we need to truncate the sequence here, which is a stupid design decision
def tokenizer(s: str):
    return token_indexer.wordpiece_tokenizer(s)[:config.max_seq_len - 2]

01/30/2019 14:30:57 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/keitakurita/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:00, 589.29it/s]
251it [00:00, 747.76it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [[UNK],
  [UNK],
  the,
  edit,
  ##s,
  made,
  under,
  my,
  user,
  ##name,
  [UNK],
  [UNK],
  [UNK],
  were,
  reverted,
  ##?,
  [UNK],
  weren,
  ##',
  ##t,
  van,
  ##dal,
  ##isms,
  ##,,
  just,
  closure,
  on,
  some,
  [UNK],
  after,
  [UNK],
  voted,
  at,
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  please,
  don,
  ##',
  ##t,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  [UNK],
  retired,
  now,
  ##.,
  ##8,
  ##9,
  ##.,
  ##20,
  ##5,
  ##.,
  ##38,
  ##.,
  ##27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.wordpiece_indexer.PretrainedBertIndexer at 0x1a2135b208>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

In [17]:
vocab = Vocabulary.from_instances(train_ds, max_vocab_size=config.max_vocab_size)

01/30/2019 14:30:59 - INFO - allennlp.data.vocabulary -   Fitting token dictionary from dataset.
100%|██████████| 267/267 [00:00<00:00, 53411.51it/s]


### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[ 101,  100,  100,  ...,    0,    0,    0],
          [ 101,  100,  100,  ..., 2001, 2130,  102],
          [ 101, 3492, 2172,  ...,    0,    0,    0],
          ...,
          [ 101, 1000,  100,  ..., 6414, 5275,  102],
          [ 101,  100,  100,  ...,  100, 2017,  102],
          [ 101,  100,  100,  ..., 4189, 2224,  102]]),
  'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
          [ 1,  2,  3,  ..., 96, 97, 98],
          [ 1,  2,  3,  ...,  0,  0,  0],
          ...,
          [ 1,  2,  3,  ..., 96, 97, 98],
          [ 1,  2,  3,  ..., 96, 97, 98],
          [ 1,  2,  3,  ..., 96, 97, 98]]),
  'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]])},
 'id': ['004b103182fb1eab',
  '0034065c7b12a7a2',
  '0015f4aa35ebe9b5',
  '008f1b06428778fe',
  '00882ab8cfa42

In [23]:
batch["tokens"]["tokens"]

tensor([[ 101,  100,  100,  ...,    0,    0,    0],
        [ 101,  100,  100,  ..., 2001, 2130,  102],
        [ 101, 3492, 2172,  ...,    0,    0,    0],
        ...,
        [ 101, 1000,  100,  ..., 6414, 5275,  102],
        [ 101,  100,  100,  ...,  100, 2017,  102],
        [ 101,  100,  100,  ..., 4189, 2224,  102]])

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 100])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True, # conserve memory
)
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                            # we'll be ignoring masks so we'll need to set this to True
                                                           allow_unmatched_keys = True)

01/30/2019 14:31:00 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/keitakurita/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
01/30/2019 14:31:00 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/keitakurita/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/hy/1czs1y5j2d58zgkqx6w_wnpw0000gn/T/tmpzzti7kvp
01/30/2019 14:31:03 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads"

In [28]:
BERT_DIM = word_embeddings.get_output_dim()

class BertSentencePooler(Seq2VecEncoder):
    def forward(self, embs: torch.tensor, 
                mask: torch.tensor=None) -> torch.tensor:
        # extract first token tensor
        return embs[:, 0]
    
    @overrides
    def get_output_dim(self) -> int:
        return BERT_DIM
    
encoder = BertSentencePooler(vocab)

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[ 101,  100,  100,  ...,    0,    0,    0],
         [ 101,  100,  100,  ..., 2001, 2130,  102],
         [ 101, 3492, 2172,  ...,    0,    0,    0],
         ...,
         [ 101, 1000,  100,  ..., 6414, 5275,  102],
         [ 101,  100,  100,  ...,  100, 2017,  102],
         [ 101,  100,  100,  ..., 4189, 2224,  102]]),
 'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ..., 96, 97, 98],
         [ 1,  2,  3,  ...,  0,  0,  0],
         ...,
         [ 1,  2,  3,  ..., 96, 97, 98],
         [ 1,  2,  3,  ..., 96, 97, 98],
         [ 1,  2,  3,  ..., 96, 97, 98]]),
 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])}

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 0.3781,  0.1085,  0.4758, -0.0077,  0.1949,  0.0161],
        [ 0.3596,  0.2196,  0.2151, -0.0192,  0.0527, -0.0243],
        [ 0.4311,  0.1409,  0.0376, -0.3367, -0.0769, -0.0157],
        [ 0.4335,  0.3701,  0.3405, -0.1184, -0.0094, -0.1064],
        [ 0.4700,  0.0739,  0.1653, -0.0565, -0.0297,  0.0323],
        [ 0.4285, -0.0113,  0.1163, -0.0126, -0.1320, -0.1505],
        [ 0.5624,  0.2789,  0.0805, -0.0140,  0.1285, -0.0224],
        [ 0.5625,  0.1163,  0.2514,  0.0678,  0.2661, -0.0518],
        [ 0.3955,  0.0895,  0.2486, -0.2068, -0.0766, -0.0138],
        [ 0.3479,  0.1997,  0.0116, -0.0916, -0.0028, -0.0203],
        [ 0.6592,  0.1787,  0.2949, -0.1487,  0.2863,  0.0260],
        [ 0.2323,  0.2659,  0.0830, -0.1042, -0.0649, -0.1018],
        [ 0.2686,  0.1543,  0.1628, -0.2537, -0.0060, -0.0945],
        [ 0.3834,  0.2561,  0.0754, -0.2584,  0.0843, -0.2079],
        [ 0.3827,  0.1597,  0.1498, -0.0688,  0.0793, -0.3707],
        [ 0.2815,  0.2058,  0.1377, -0.1

In [36]:
model(**batch)

{'class_logits': tensor([[ 0.5553,  0.0279,  0.2647, -0.0261, -0.1336, -0.0631],
         [ 0.2300,  0.2907,  0.3286, -0.0085, -0.1682, -0.1022],
         [ 0.3542,  0.2422,  0.0602, -0.2695, -0.0081, -0.0589],
         [ 0.5020,  0.1734,  0.4356,  0.0111,  0.3813, -0.2373],
         [ 0.3623,  0.1063,  0.3194, -0.1453,  0.0744, -0.0130],
         [ 0.3431,  0.0989,  0.0216, -0.1020, -0.1255,  0.0649],
         [ 0.3838,  0.1365,  0.0379, -0.0341,  0.1253, -0.0776],
         [ 0.4509,  0.1045,  0.2311, -0.1048,  0.0933, -0.0869],
         [ 0.3045,  0.0623,  0.0757, -0.1947, -0.0396, -0.0552],
         [ 0.4100,  0.2782,  0.1424, -0.2658,  0.0340, -0.1136],
         [ 0.4522,  0.1980, -0.0256, -0.2547, -0.0491, -0.1085],
         [ 0.3956,  0.0516,  0.1346, -0.0352, -0.1077, -0.0150],
         [ 0.1833,  0.0885,  0.2918, -0.0740, -0.0278, -0.0750],
         [ 0.2402,  0.3118,  0.0473, -0.1252,  0.0389, -0.1903],
         [ 0.4211,  0.2523,  0.2994, -0.0343,  0.1355, -0.1534],
         

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.7450, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[]

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

01/30/2019 14:31:49 - INFO - allennlp.training.trainer -   Beginning training.
01/30/2019 14:31:49 - INFO - allennlp.training.trainer -   Epoch 0/1
01/30/2019 14:31:50 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 2061.193216
01/30/2019 14:31:50 - INFO - allennlp.training.trainer -   Training
loss: 0.7018 ||: 100%|██████████| 5/5 [00:44<00:00,  7.12s/it]
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -                     Training |  Validation
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -   cpu_memory_MB |  2061.193  |       N/A
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -   loss          |     0.702  |       N/A
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -   Epoch duration: 00:00:44
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:00:44
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -   Epoch 1/1
01/30/2019 14:32:34 - INFO - allennlp.training.trainer -   Peak CPU memor